# Part 3: Static Replication

In [31]:
import numpy as np
from scipy.stats import norm
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from scipy.stats import norm
from scipy.optimize import brentq
from scipy.integrate import quad

 # <a id = "top">Contents</a>

# [0. Preparing Work](#p1)
# [1. Payoff Function](#p2)
# [2. “Model-free” Integrated Variance](#p3)

#  <a id = "p1"> 0.<font color = "green"> Preparing Work [(back to contents)](#top)

### Step 1: Import data

In [3]:
discount = pd.read_csv('zero_rates_20201201.csv')
spx_df2 = pd.read_csv('spx_df2.csv')
spy_df2 = pd.read_csv('spy_df2y.csv')
S0_spx = 3662.45
S0_spy = 366.02
T2 = 0.1232876712328767
r2 = 0.0020510755555555554

# Results obtain from Part 2
alpha_spx = 1.81650
beta_spx = 0.7
rho_spx = -0.40430
nu_spx = 2.79016

alpha_spy = 0.90813
beta_spy = 0.7
rho_spy = -0.48878
nu_spy = 2.72852

### Step 2: Find ATM data

#### SPX

In [4]:
spx_call = spx_df2
spx_call = spx_df2[spx_df2['cp_flag'] == 'C']
spx_call = spx_call[spx_call['strike_price']*0.001 >= S0_spx]

In [5]:
spx_put = spx_df2
spx_put = spx_df2[spx_df2['cp_flag'] == 'P']
spx_put = spx_put[spx_put['strike_price']*0.001 <= S0_spx]

In [6]:
spx = pd.concat([spx_call, spx_put])
spx = spx.sort_values(by = 'vols_bs', ascending = False)

#### SPY

In [7]:
spy_call = spy_df2
spy_call = spy_df2[spy_df2['cp_flag'] == 'C']
spy_call = spy_call[spy_call['strike'] >= S0_spy]

In [8]:
spy_put = spy_df2
spy_put = spy_df2[spy_df2['cp_flag'] == 'P']
spy_put = spy_put[spy_put['strike'] <= S0_spy]

In [9]:
spy = pd.concat([spy_call, spy_put])
spy = spy.sort_values(by = 'vols_bs', ascending = False)

### Step 3: Introduce Model and Other Defined Functions

#### Black Scholes Model

In [19]:
# Call Option
def BS_Call(r, S0, K, T, sigma):
    discount_factor = np.exp(-r*T)
    d1 = (np.log(S0/K) + (r + (sigma**2)/2)*T) /(sigma*np.sqrt(T))  
    d2 = d1 - sigma*np.sqrt(T)
    price = S0 * norm.cdf(d1, loc = 0, scale = 1) - K * discount_factor * norm.cdf(d2, loc = 0, scale = 1)
    return price

# Put Option
def BS_Put(r, S0, K, T, sigma):
    discount_factor = np.exp(-r*T)
    d1 = (np.log(S0/K) + (r + (sigma**2)/2)*T) /(sigma*np.sqrt(T))  
    d2 = d1 - sigma*np.sqrt(T)
    price = K * discount_factor * norm.cdf(-d2, loc = 0, scale = 1) - S0 * norm.cdf(-d1, loc = 0, scale = 1)
    return price

# “Model-free” integrated variance
def BS_callintegrand(r, S, K, T, sigma):
    price = BS_Call(r, S, K, T, sigma) / K**2
    return price

def BS_putintegrand(r, S, K, T, sigma):
    price = BS_Put(r, S, K, T, sigma) / K**2
    return price

#### Bachelier Model

In [36]:
Bachelier_Call(r2, S0_spx, 3660, T2, 688)

97.57893290325879

In [35]:
BS_Call(r2, S0_spx, 3660, T2, 0.1889)

98.54574816202489

In [34]:
spx

,date,exdate,cp_flag,strike_price,best_bid,best_offer,exercise_style,mid_price,strike,payoff,vols_bs,Difference
553,20201201,20210115,P,3660000,94.8,95.5,E,95.15,3660.0,put,0.188860,2.45
185,20201201,20210115,C,3665000,91.5,92.4,E,91.95,3665.0,call,0.180798,2.55
552,20201201,20210115,P,3655000,92.8,93.5,E,93.15,3655.0,put,0.189871,7.45
186,20201201,20210115,C,3670000,88.6,89.5,E,89.05,3670.0,call,0.179825,7.55
551,20201201,20210115,P,3650000,90.8,91.6,E,91.20,3650.0,put,0.190902,12.45
...,...,...,...,...,...,...,...,...,...,...,...,...
270,20201201,20210115,P,500000,0.0,0.1,E,0.05,500.0,put,1.636102,3162.45
269,20201201,20210115,P,400000,0.0,0.1,E,0.05,400.0,put,1.821174,3262.45
268,20201201,20210115,P,300000,0.0,0.1,E,0.05,300.0,put,2.062602,3362.45
267,20201201,20210115,P,200000,0.0,0.1,E,0.05,200.0,put,2.408988,3462.45


In [20]:
# Call Option
def Bachelier_Call(r, S0, K, T, sigma):
    discount_factor = np.exp(-r*T)
    c = (S0-K) / (sigma*np.sqrt(T))
    price = discount_factor * ((S0-K)*norm.cdf(c, loc = 0, scale = 1) + sigma * np.sqrt(T)*norm.pdf(c))
    return price

# Put Option
def Bachelier_Put(r, S0, K, T, sigma):
    discount_factor = np.exp(-r*T)
    c = (S0 - K) / (sigma*np.sqrt(T))
    price = discount_factor * ((K-S0)*norm.cdf(-c, loc = 0, scale = 1) + sigma * np.sqrt(T)*norm.pdf(-c))
    return price

# “Model-free” integrated variance
def bachelier_callintegrand(r, S, K, T, sigma):
    price = Bachelier_Call(r, S, K, T, sigma) / K**2
    return price

def bachelier_putintegrand(r, S, K, T, sigma):
    price = Bachelier_Put(r, S, K, T, sigma) / K**2
    return price

#### SABR Model

In [21]:
# Implied Volatility
def SABR(F, K, T, alpha, beta, rho, nu):
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom

    return sabrsigma

# Call Option
def SABRCall(r, S, K, T, alpha, beta, rho, nu):
    sabr_vol = SABR(S*np.exp(r*T), K, T, alpha, beta, rho, nu)
    return BS_Call(r, S, K, T, sabr_vol)

# Put Option
def SABRPut(r, S, K, T, alpha, beta, rho, nu):
    sabr_vol = SABR(S*np.exp(r*T), K, T, alpha, beta, rho, nu)
    return BS_Put(r, S, K, T, sabr_vol)

# “Model-free” integrated variance
def sabrcallintegrand(r, S, K, T, alpha, beta, rho, nu):
    price = SABRCall(r, S, K, T, alpha, beta, rho, nu) / K**2
    return price

def sabrputintegrand(r, S, K, T, alpha, beta, rho, nu):
    price = SABRPut(r, S, K, T, alpha, beta, rho, nu) / K**2
    return price

### Step 4: Obtain Volatility

> First we find the smallest differences between strike price and S0.

In [11]:
spx['Difference'] = abs(spx['strike'] - S0_spx)
spx = spx.sort_values(by = 'Difference')

In [12]:
spy['Difference'] = abs(spy['strike'] - S0_spy)
spy = spy.sort_values(by = 'Difference')

>Then we can obtain the volatility of Black Scholes Model.

In [13]:
Sigma_LN_spx = spx.iloc[0]['vols_bs'] 
Sigma_LN_spy = spy.iloc[0]['vols_bs'] 
print(f'Sigma_LN_spx = {Sigma_LN_spx:.4f}, Sigma_LN_spx = {Sigma_LN_spy:.4f}')

Sigma_LN_spx = 0.1889, Sigma_LN_spx = 0.1978


> After that, we obtain the volatility of Bachelier Model.\
Since ATM the volatility of Black Scholes and Bachelier are the same, then we just let Sigma_N_x = x.iloc[0]['vols_bachelier'].

In [14]:
Sigma_N_spx = spx.iloc[0]['vols_bs'] 
Sigma_N_spy = spy.iloc[0]['vols_bs'] 
print(f'Sigma_N_spx = {Sigma_N_spx:.4f}, Sigma_N_spy = {Sigma_N_spy:.4f}')

Sigma_N_spx = 0.1889, Sigma_N_spy = 0.1978


In [15]:
sigma = {'Sigma_LN': [Sigma_LN_spx, Sigma_LN_spy],
        'Sigma_N': [Sigma_N_spx, Sigma_N_spy]}

sigma = pd.DataFrame(sigma, index=['SPX', 'SPY'])
sigma = sigma.T
sigma

,SPX,SPY
Sigma_LN,0.18886,0.197767
Sigma_N,0.18886,0.197767


### Finally, we can start to do the questions now.

#  <a id = "p2"> 1.<font color = "green">Payoff Function [(back to contents)](#top)

Given payoff function:
$$
h(S_T) = S^{1/3}_T + 1.5 \cdot \log(S_T) + 10.0
$$

Then, first derivative:
$$
h'(S_T) = \frac{1}{3} \cdot S^{-\frac{2}{3}}_T + \frac{3}{2} \cdot S^{-1}_T
$$

Second derivative:
$$
h''(S_T) = -\frac{2}{9} \cdot S^{-\frac{5}{3}}_T - \frac{3}{2} \cdot S^{-2}_T
$$

> Define a function.

In [16]:
def h(K):
    if abs(K) < 1e-12:
        K = 1e-12
    return K**(1/3) + 1.5 * np.log(K) + 10

def h_dif2(K):
    if abs(K) < 1e-12:
        K = 1e-12
    return -2/9 * K**(-5/3) - 3/2 * K**(-2)

> Obtain forward price.

In [17]:
F_spx = S0_spx * np.exp(r2 * T2)
F_spy = S0_spy * np.exp(r2 * T2)

Note that:
\begin{equation*}
    \begin{split}
      V_0 = e^{-rT}h(F) + \underbrace{\int_0^{F}h''(K)P(K)\;dK}_{\mbox{put integral}} + \underbrace{\int_{F}^{\infty}h''(K)C(K)\;dK}_{\mbox{call integral}}\\
    \end{split}
\end{equation*}


> Black Scholes Model.

In [22]:
SPX_BS = np.exp(-r2*T2) * h(F_spx) + h_dif2(F_spx) * BS_callintegrand(r2, S0_spx, F_spx, T2, Sigma_LN_spx) + h_dif2(F_spx) * BS_putintegrand(r2, S0_spx, F_spx, T2, Sigma_LN_spx)
SPY_BS = np.exp(-r2*T2) * h(F_spy) + h_dif2(F_spy) * BS_callintegrand(r2, S0_spy, F_spy, T2, Sigma_LN_spy) + h_dif2(F_spy) * BS_putintegrand(r2, S0_spy, F_spy, T2, Sigma_LN_spy)
print(f'SPX_BS = {SPX_BS:.4f}, SPY_BS = {SPY_BS:.4f}')

SPX_BS = 37.7153, SPY_BS = 26.0017


> Bachelier Model.

In [23]:
SPX_Bachelier = np.exp(-r2*T2) * h(F_spx) + h_dif2(F_spx) * bachelier_callintegrand(r2, S0_spx, F_spx, T2, Sigma_N_spx) + h_dif2(F_spx) * bachelier_putintegrand(r2, S0_spx, F_spx, T2, Sigma_N_spx)
SPY_Bachelier = np.exp(-r2*T2) * h(F_spy) + h_dif2(F_spy) * bachelier_callintegrand(r2, S0_spy, F_spy, T2, Sigma_LN_spy) + h_dif2(F_spy) * bachelier_putintegrand(r2, S0_spy, F_spy, T2, Sigma_N_spy)
print(f'SPX_Bachelier = {SPX_Bachelier:.4f}, SPY_Bachelier = {SPY_Bachelier:.4f}')

SPX_Bachelier = 37.7153, SPY_Bachelier = 26.0017


>SABR Model.

In [24]:
SPX_sabr = np.exp(-r2*T2) * h(F_spx) + h_dif2(F_spx) * sabrcallintegrand(r2, S0_spx, F_spx, T2, alpha_spx, beta_spx, rho_spx, nu_spx) + h_dif2(F_spx) * sabrputintegrand(r2, S0_spx, F_spx, T2, alpha_spx, beta_spx, rho_spx, nu_spx)
SPY_sabr = np.exp(-r2*T2) * h(F_spy) + h_dif2(F_spy) * sabrcallintegrand(r2, S0_spy, F_spy, T2, alpha_spy, beta_spy, rho_spy, nu_spy) + h_dif2(F_spy) * sabrputintegrand(r2, S0_spy, F_spy, T2, alpha_spy, beta_spy, rho_spy, nu_spy)
print(f'SPX_sabr = {SPX_sabr:.4f}, SPY_sabr = {SPY_sabr:.4f}')

SPX_sabr = 37.7153, SPY_sabr = 26.0017


> Summary Table.

In [25]:
Summary_payoff = pd.DataFrame({'SPX' : [SPX_BS,
                                               SPX_Bachelier,
                                               SPX_sabr],
                                 'SPY' : [SPY_BS,
                                               SPY_Bachelier,
                                               SPY_sabr]}).T
Summary_payoff.columns = ['Black-Scholes', 'Bachelier', 'SABR']
Summary_payoff

,Black-Scholes,Bachelier,SABR
SPX,37.715275,37.715275,37.715275
SPY,26.001659,26.001659,26.001659


#  <a id = "p3"> 2.<font color = "green">“Model-free” Integrated Variance [(back to contents)](#top)

$$
\sigma^2_{\text{MF}} T = \mathbb{E}\left[\int_{0}^{T} \sigma^2_t \, dt\right]
$$

> Black Scholes Model.

In [26]:
# SPX
E_var_SPX_BS = Sigma_LN_spx **2 * T2
MF_SPX_BS = np.sqrt(E_var_SPX_BS/T2)

#SPY
E_var_SPY_BS = Sigma_LN_spy **2 * T2
MF_SPY_BS = np.sqrt(E_var_SPY_BS/T2)

print(f'Variance: E_var_SPX_BS = {E_var_SPX_BS:.4f}, E_var_SPY_BS = {E_var_SPY_BS:.4f}')
print(f'Sigma: MF_SPX_BS = {MF_SPX_BS:.4f}, MF_SPY_BS = {MF_SPY_BS:.4f}')

Variance: E_var_SPX_BS = 0.0044, E_var_SPY_BS = 0.0048
Sigma: MF_SPX_BS = 0.1889, MF_SPY_BS = 0.1978


> Bachelier Model.

In [27]:
# SPX
E_var_SPX_bachelier = Sigma_N_spx **2 * T2
MF_SPX_bachelier = np.sqrt(E_var_SPX_BS/T2)

#SPY
E_var_SPY_bachelier = Sigma_N_spy **2 * T2
MF_SPY_bachelier = np.sqrt(E_var_SPY_BS/T2)

print(f'Variance: E_var_SPX_bachelier = {E_var_SPX_bachelier:.4f}, E_var_SPY_bachelier = {E_var_SPY_bachelier:.4f}')
print(f'Sigma: MF_SPX_bachelier = {MF_SPX_bachelier:.4f}, MF_SPY_bachelier = {MF_SPY_bachelier:.4f}')

Variance: E_var_SPX_bachelier = 0.0044, E_var_SPY_bachelier = 0.0048
Sigma: MF_SPX_bachelier = 0.1889, MF_SPY_bachelier = 0.1978


> SABR Model.

In [28]:
#SPX
Sabr_put_spx = quad(lambda x: sabrputintegrand(r2, S0_spx, x, T2, alpha_spx, beta_spx, rho_spx, nu_spx), 1e-6, F_spx)
Sabr_call_spx = quad(lambda x: sabrcallintegrand(r2, S0_spx, x, T2, alpha_spx, beta_spx, rho_spx, nu_spx), F_spx, 5000)
E_var_sabr_Spx = 2*np.exp(r2*T2)*(Sabr_put_spx[0] + Sabr_call_spx[0])
MF_sabr_Spx = np.sqrt(E_var_sabr_Spx/T2)

#SPY
Sabr_put_spy = quad(lambda x: sabrputintegrand(r2, S0_spy, x, T2, alpha_spy, beta_spy, rho_spy, nu_spy), 1e-6, F_spy)
Sabr_call_spy = quad(lambda x: sabrcallintegrand(r2, S0_spy, x, T2, alpha_spy, beta_spy, rho_spy, nu_spy), F_spy, 5000)
E_var_sabr_Spy = 2*np.exp(r2*T2)*(Sabr_put_spy[0] + Sabr_call_spy[0])
MF_sabr_Spy = np.sqrt(E_var_sabr_Spy/T2)

print(f'Variance: E_var_sabr_Spx = {E_var_sabr_Spx:.4f}, E_var_sabr_Spy = {E_var_sabr_Spy:.4f}')
print(f'Sigma: MF_sabr_Spx = {MF_sabr_Spx:.4f}, MF_sabr_Spy = {MF_sabr_Spy:.4f}')

Variance: E_var_sabr_Spx = 0.0063, E_var_sabr_Spy = 0.0060
Sigma: MF_sabr_Spx = 0.2267, MF_sabr_Spy = 0.2209


> Summary Table.

In [29]:
sigma_Model_Free = pd.DataFrame({'SPX T=45' : [MF_SPX_BS,
                                               MF_SPX_bachelier,
                                               MF_sabr_Spx],
                                 'SPY T=45' : [MF_SPY_BS,
                                               MF_SPY_bachelier,
                                               MF_sabr_Spy]}).T
sigma_Model_Free.columns = ['Black-Scholes', 'Bachelier', 'SABR']
sigma_Model_Free

,Black-Scholes,Bachelier,SABR
SPX T=45,0.188860,0.188860,0.226675
SPY T=45,0.197767,0.197767,0.220901


In [30]:
variance = pd.DataFrame({'SPX T=45' : [E_var_SPX_BS,
                                       E_var_SPX_bachelier,
                                       E_var_sabr_Spx],
                         'SPY T=45' : [E_var_SPY_BS,
                                       E_var_SPY_bachelier,
                                       E_var_sabr_Spy]}).T
variance.columns = ['Black-Scholes', 'Bachelier', 'SABR']
variance

,Black-Scholes,Bachelier,SABR
SPX T=45,0.004397,0.004397,0.006335
SPY T=45,0.004822,0.004822,0.006016
